***
## __Imports__
***

In [35]:
# -*- coding: utf-8 -*-
import os

import pandas as pd

from aqmd_pylib.aqmd_lib import data_toolkit as dtk
from aqmd_pylib.aqmd_lib import util
from aqmd_pylib.aqmd_lib.util import nLoop

from PyScripts import InnovCorridor_CodeKit as ic

***
## __Settings and File Paths__
***

In [36]:
SAVE_TABLES = True
SAVE_GRAPHS = False

fPath_carb = r'../data/~originals/CARB/RUBIDOUX_PM25HR_PICKDATA_2021-10-31.csv'
fPath_historicalClarity = r'../data/~originals/Clarity_Originals/historical_pm25cal_highres.csv'
fPath_metadata = r'../data/metadata/metadata.xlsx'
fPath_weatherData = r'..\data\~originals\OpenWeather Data\OpenWeather_September_2020_to_September_2021.csv'
fpath_no2 = r'../data/~originals/Clarity_Originals/ucr_no2_unaveraged.csv'

FPath_analysis = r'..\Analysis'
FPath_travelTime_csv = r'..\data\~originals\travel_time_csv'
FPath_clarity_originals_HighRes = r'../data/~originals/Clarity_Originals/HighRes'
FPath_clarity_originals_hourly = r'../data/~originals/Clarity_Originals/Hourly'
FPath_PEMS = '../data/~originals/pems_freeway/'
FPath_GridSmart_csv = '../data/~originals/Gridsmart_originals/gridsmart_csv/'

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

***
## __Metadata__
***

In [37]:
clarity_metadata = pd.read_excel(fPath_metadata, sheet_name='ClaritySensors', header=0)
clarity_metadata

,sensorID,nickname,Cross Streets,Latitude,Longitude,Cardinal Relation
0,ARHF78W6,Magnolia,Magnolia & Tyler,33.911928,-117.461394,NaN
1,AY3HFT4R,Iowa,Iowa & University,33.975591,-117.339897,NaN
2,AGJ1PPLJ,Chicago,Chicago & University,33.975566,-117.348524,NaN
3,A4T789S6,Cranford,Cranford & University,33.975624,-117.344182,NaN


In [38]:
travelPath_metadata = pd.read_excel(fPath_metadata, sheet_name='TravelPaths', header=0)
travelPath_metadata

,pathID,Name,Distance(m),Start Node,Start Addreess,Start Latitude,Start Longitude,End Node,End Address,End Latitude,End Longitude,Path Info,Cardinal Relation
0,HEW1,Local 1,836,H6,"948 University Ave, Riverside, CA 92507",33.97580,-117.33385,H5,"1426 University Ave, Riverside, CA 92507",33.97570,-117.34262,WB Innovation Corridor,NaN
1,HEW2,Local 2,893,H5,"1426 University Ave, Riverside, CA 92507",33.97570,-117.34262,H4,"1889 University Ave STE 109, Riverside, CA 92507",33.97614,-117.35254,WB Innovation Corridor,NaN
2,HEW3,Local 3,829,H4,"1889 University Ave STE 109, Riverside, CA 92507",33.97614,-117.35254,H3,"2460 University Ave, Riverside, CA 92507",33.97574,-117.35254,WB Innovation Corridor,NaN
3,HEW4,Local 4,490,H3,"2460 University Ave, Riverside, CA 92507",33.97574,-117.35254,H2,"2980 University Ave, Riverside, CA 92507",33.97768,-117.36593,WB Innovation Corridor,NaN
4,HEW5,Local 5,929,H2,"2980 University Ave, Riverside, CA 92507",33.97768,-117.36593,H1,"Cesar E. Chavez Memorial, 3746 University Ave, Riverside, CA 92501",33.98211,-117.37477,WB Innovation Corridor,NaN
5,HWE1,Local 6,931,H1,"Cesar E. Chavez Memorial, 3746 University Ave, Riverside, CA 92501",33.98211,-117.37477,H2,"2980 University Ave, Riverside, CA 92507",33.97768,-117.36593,EB Innnovation Corridor,NaN
6,HWE2,Local 7,491,H2,"2980 University Ave, Riverside, CA 92507",33.97768,-117.36593,H3,"2460 University Ave, Riverside, CA 92507",33.97574,-117.35254,EB Innnovation Corridor,NaN
7,HWE3,Local 8,808,H3,"2460 University Ave, Riverside, CA 92507",33.97574,-117.35254,H4,"1889 University Ave STE 109, Riverside, CA 92507",33.97614,-117.35254,EB Innnovation Corridor,NaN
8,HWE4,Local 9,918,H4,"1889 University Ave STE 109, Riverside, CA 92507",33.97614,-117.35254,H5,"1426 University Ave, Riverside, CA 92507",33.97570,-117.34262,EB Innnovation Corridor,NaN
9,HWE5,Local 10,836,H5,"1426 University Ave, Riverside, CA 92507",33.97570,-117.34262,H6,"948 University Ave, Riverside, CA 92507",33.97580,-117.33385,EB Innnovation Corridor,NaN


In [39]:
travelNode_metadata = pd.read_excel(fPath_metadata, sheet_name='TravelNodes', header=0)
travelNode_metadata

,nodeID,Address,Latitude,Longitude
0,H1,"Cesar E. Chavez Memorial, 3746 University Ave, Riverside, CA 92501",33.98211,-117.37477
1,H2,"2980 University Ave, Riverside, CA 92507",33.97768,-117.36593
2,H3,"2460 University Ave, Riverside, CA 92507",33.97574,-117.35254
3,H4,"1889 University Ave STE 109, Riverside, CA 92507",33.97614,-117.35254
4,H5,"1426 University Ave, Riverside, CA 92507",33.97570,-117.34262
5,H6,"948 University Ave, Riverside, CA 92507",33.97580,-117.33385
6,F1,"888 Martin Luther King Blvd, Riverside, CA 92507",33.96839,-117.33035
7,F2,"ARCO, 1360 W Blaine St, Riverside, CA 92507",33.98279,-117.34147
8,F3,"3219 Spruce St, Riverside, CA 92501",33.99191,-117.36048
9,F4,"Subway, 3315 14th St, Riverside CA 92501",33.97471,-117.37386


***
# __Background Concentration__
***

***
> ## __CARB Data__
***

In [40]:
carb_pm25 = pd.read_csv(fPath_carb)
carb_pm25 = ic.carb_PM25(carb_pm25)
carb_pm25_data = carb_pm25.get_data()

carb_pm25_data.info()
carb_pm25_data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6662 entries, 2021-01-01 00:00:00-08:00 to 2021-10-31 23:00:00-07:00
Data columns (total 1 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   PM2.5 Background [ug/m3]  6662 non-null   float64
dtypes: float64(1)
memory usage: 104.1 KB


,PM2.5 Background [ug/m3]
datetime-America/Los_Angeles,
2021-01-01 00:00:00-08:00,2.0
2021-01-01 01:00:00-08:00,1.0
2021-01-01 02:00:00-08:00,-2.0
2021-01-01 03:00:00-08:00,-1.0
2021-01-01 04:00:00-08:00,1.0
...,...
2021-10-31 19:00:00-07:00,8.0
2021-10-31 20:00:00-07:00,17.0
2021-10-31 21:00:00-07:00,19.0


In [41]:
carb_pm25_data_interp = carb_pm25_data.resample('15T')
carb_pm25_data_interp

In [42]:
carb_pm25_data_interp = carb_pm25_data_interp.ffill()
carb_pm25_data_interp = carb_pm25_data_interp[~carb_pm25_data_interp.index.duplicated()]
if SAVE_TABLES:
    carb_pm25_data_interp.to_csv('../data/CARB/Rubidoux_Background_pm25.csv')
carb_pm25_data_interp.info()
carb_pm25_data_interp.head(len(carb_pm25_data_interp))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29177 entries, 2021-01-01 00:00:00-08:00 to 2021-10-31 23:00:00-07:00
Freq: 15T
Data columns (total 1 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   PM2.5 Background [ug/m3]  29177 non-null  float64
dtypes: float64(1)
memory usage: 455.9 KB


,PM2.5 Background [ug/m3]
datetime-America/Los_Angeles,
2021-01-01 00:00:00-08:00,2.0
2021-01-01 00:15:00-08:00,2.0
2021-01-01 00:30:00-08:00,2.0
2021-01-01 00:45:00-08:00,2.0
2021-01-01 01:00:00-08:00,1.0
...,...
2021-10-31 22:00:00-07:00,16.0
2021-10-31 22:15:00-07:00,16.0
2021-10-31 22:30:00-07:00,16.0


***
# Hyperlocals Emmissions (Clarity)__
***

***
> ### __Historical Data__
***

In [43]:
historical_clarity = pd.read_csv(fPath_historicalClarity)
historical_clarity = ic.clarity_historical(historical_clarity)
historical_clarity_data = historical_clarity.get_data()

historical_clarity_data.info()
historical_clarity_data.head(len(historical_clarity_data))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 88847 entries, 2020-09-29 12:05:00-07:00 to 2021-06-22 23:58:00-07:00
Data columns (total 9 columns):
 #   Column                                                          Non-Null Count  Dtype  
---  ------                                                          --------------  -----  
 0   Sensor Name                                                     88847 non-null  object 
 1   Intersection                                                    88847 non-null  object 
 2   Device ID                                                       88847 non-null  object 
 3   Date                                                            88847 non-null  object 
 4   PM2.5 Highest Resolution Mass Concentration Calibrated [ug/m3]  88847 non-null  float64
 5   Rel. Humidity Internal Highest Resolution [%]                   88847 non-null  float64
 6   Temperature Internal Highest Resolution [degC]                  88847 non-null  float64
 7   Lat

,Sensor Name,Intersection,Device ID,Date,PM2.5 Highest Resolution Mass Concentration Calibrated [ug/m3],Rel. Humidity Internal Highest Resolution [%],Temperature Internal Highest Resolution [degC],Latitude,Longitude
datetime-America/Los_Angeles,,,,,,,,,
2020-09-29 12:05:00-07:00,Magnolia,Magnolia & Tyler,ARHF78W6,9/29/2020 12:05,3.573135,25.265888,34.912262,33.911928,-117.461394
2020-09-29 13:06:00-07:00,Magnolia,Magnolia & Tyler,ARHF78W6,9/29/2020 13:06,2.654986,22.885481,38.290230,33.911928,-117.461394
2020-09-29 13:24:00-07:00,Magnolia,Magnolia & Tyler,ARHF78W6,9/29/2020 13:24,2.318490,22.088961,38.768219,33.911928,-117.461394
2020-09-29 13:41:00-07:00,Magnolia,Magnolia & Tyler,ARHF78W6,9/29/2020 13:41,2.023389,23.118944,38.981842,33.911928,-117.461394
2020-09-29 13:58:00-07:00,Magnolia,Magnolia & Tyler,ARHF78W6,9/29/2020 13:58,1.755854,21.947052,39.166092,33.911928,-117.461394
...,...,...,...,...,...,...,...,...,...
2021-06-22 23:40:00-07:00,Chicago,Chicago & University,AGJ1PPLJ,6/22/2021 23:40,7.240612,52.887772,23.197525,33.975566,-117.348524
2021-06-22 23:44:00-07:00,Magnolia,Magnolia & Tyler,ARHF78W6,6/22/2021 23:44,6.212309,56.778820,22.420464,33.911928,-117.461394
2021-06-22 23:50:00-07:00,Iowa,Iowa & University,AY3HFT4R,6/22/2021 23:50,6.210450,52.457466,23.152130,33.975591,-117.339897


***
> ### __Original Clarity Data__
***

In [44]:
clarity_original_HighRes = util.csv_folder2table(FPath_clarity_originals_HighRes)
clarity_HighRes = ic.clarity_original(clarity_original_HighRes)
clarity_HighRes_data = clarity_HighRes.get_data()
clarity_HighRes_data.info()
clarity_HighRes_data.head(len(clarity_original_HighRes))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 83608 entries, 2021-01-01 00:07:14.361000-08:00 to 2021-07-10 23:58:39.211000-07:00
Data columns (total 10 columns):
 #   Column                                                          Non-Null Count  Dtype              
---  ------                                                          --------------  -----              
 0   Sensor Name                                                     83608 non-null  object             
 1   Intersection                                                    83608 non-null  object             
 2   Device ID                                                       83608 non-null  object             
 3   datetime-UTC                                                    83608 non-null  datetime64[ns, UTC]
 4   PM2.5 Highest Resolution Mass Concentration Calibrated [ug/m3]  63278 non-null  float64            
 5   NO2 Highest Resolution Concentration Calibrated [ppb]           0 non-null      float64   

,Sensor Name,Intersection,Device ID,datetime-UTC,PM2.5 Highest Resolution Mass Concentration Calibrated [ug/m3],NO2 Highest Resolution Concentration Calibrated [ppb],Temperature Internal Highest Resolution [degC],Rel. Humidity Internal Highest Resolution [%],Latitude,Longitude
datetime-America/Los_Angeles,,,,,,,,,,
2021-01-01 00:07:14.361000-08:00,Iowa,Iowa & University,AY3HFT4R,2021-01-01 08:07:14.361000+00:00,NaN,NaN,15.00,29.89,33.975591,-117.339897
2021-01-01 00:09:41.191000-08:00,Magnolia,Magnolia & Tyler,ARHF78W6,2021-01-01 08:09:41.191000+00:00,NaN,NaN,15.99,28.41,33.911928,-117.461394
2021-01-01 00:11:40.746000-08:00,Cranford,Cranford & University,A4T789S6,2021-01-01 08:11:40.746000+00:00,NaN,NaN,15.07,29.28,33.975624,-117.344182
2021-01-01 00:12:36.634000-08:00,Chicago,Chicago & University,AGJ1PPLJ,2021-01-01 08:12:36.634000+00:00,NaN,NaN,15.27,29.02,33.975566,-117.348524
2021-01-01 00:24:25.849000-08:00,Iowa,Iowa & University,AY3HFT4R,2021-01-01 08:24:25.849000+00:00,NaN,NaN,14.91,29.69,33.975591,-117.339897
...,...,...,...,...,...,...,...,...,...,...
2021-07-10 23:55:03.070000-07:00,Chicago,Chicago & University,AGJ1PPLJ,2021-07-11 06:55:03.070000+00:00,9.01,NaN,25.78,41.70,33.975566,-117.348524
2021-07-10 23:57:16.538000-07:00,Magnolia,Magnolia & Tyler,ARHF78W6,2021-07-11 06:57:16.538000+00:00,10.61,NaN,25.62,41.46,33.911928,-117.461394
2021-07-10 23:57:30.107000-07:00,Cranford,Cranford & University,A4T789S6,2021-07-11 06:57:30.107000+00:00,8.43,NaN,25.46,42.16,33.975624,-117.344182


In [45]:
clarity_HighRes_data = clarity_HighRes.get_data()
clarity_HighRes_data.info()
clarity_HighRes_data.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 83608 entries, 2021-01-01 00:07:14.361000-08:00 to 2021-07-10 23:58:39.211000-07:00
Data columns (total 10 columns):
 #   Column                                                          Non-Null Count  Dtype              
---  ------                                                          --------------  -----              
 0   Sensor Name                                                     83608 non-null  object             
 1   Intersection                                                    83608 non-null  object             
 2   Device ID                                                       83608 non-null  object             
 3   datetime-UTC                                                    83608 non-null  datetime64[ns, UTC]
 4   PM2.5 Highest Resolution Mass Concentration Calibrated [ug/m3]  63278 non-null  float64            
 5   NO2 Highest Resolution Concentration Calibrated [ppb]           0 non-null      float64   

,Sensor Name,Intersection,Device ID,datetime-UTC,PM2.5 Highest Resolution Mass Concentration Calibrated [ug/m3],NO2 Highest Resolution Concentration Calibrated [ppb],Temperature Internal Highest Resolution [degC],Rel. Humidity Internal Highest Resolution [%],Latitude,Longitude
datetime-America/Los_Angeles,,,,,,,,,,
2021-01-01 00:07:14.361000-08:00,Iowa,Iowa & University,AY3HFT4R,2021-01-01 08:07:14.361000+00:00,NaN,NaN,15.00,29.89,33.975591,-117.339897
2021-01-01 00:09:41.191000-08:00,Magnolia,Magnolia & Tyler,ARHF78W6,2021-01-01 08:09:41.191000+00:00,NaN,NaN,15.99,28.41,33.911928,-117.461394
2021-01-01 00:11:40.746000-08:00,Cranford,Cranford & University,A4T789S6,2021-01-01 08:11:40.746000+00:00,NaN,NaN,15.07,29.28,33.975624,-117.344182
2021-01-01 00:12:36.634000-08:00,Chicago,Chicago & University,AGJ1PPLJ,2021-01-01 08:12:36.634000+00:00,NaN,NaN,15.27,29.02,33.975566,-117.348524
2021-01-01 00:24:25.849000-08:00,Iowa,Iowa & University,AY3HFT4R,2021-01-01 08:24:25.849000+00:00,NaN,NaN,14.91,29.69,33.975591,-117.339897


***
> ### __Clarity PM2.5 Measurements__
***

In [46]:
clarity_HighRes_pm25 = clarity_HighRes.get_PM25()
clarity_HighRes_pm25.info()
clarity_HighRes_pm25.head(len(clarity_HighRes_pm25))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 63278 entries, 2021-03-02 12:39:24.550000-08:00 to 2021-07-10 23:58:39.211000-07:00
Data columns (total 8 columns):
 #   Column                                                          Non-Null Count  Dtype  
---  ------                                                          --------------  -----  
 0   Sensor Name                                                     63278 non-null  object 
 1   Intersection                                                    63278 non-null  object 
 2   Device ID                                                       63278 non-null  object 
 3   PM2.5 Highest Resolution Mass Concentration Calibrated [ug/m3]  63278 non-null  float64
 4   Temperature Internal Highest Resolution [degC]                  63278 non-null  float64
 5   Rel. Humidity Internal Highest Resolution [%]                   63278 non-null  float64
 6   Latitude                                                        63278 non-null  fl

,Sensor Name,Intersection,Device ID,PM2.5 Highest Resolution Mass Concentration Calibrated [ug/m3],Temperature Internal Highest Resolution [degC],Rel. Humidity Internal Highest Resolution [%],Latitude,Longitude
datetime-America/Los_Angeles,,,,,,,,
2021-03-02 12:39:24.550000-08:00,Cranford,Cranford & University,A4T789S6,11.81,26.84,9.31,33.975624,-117.344182
2021-03-02 12:42:19.270000-08:00,Magnolia,Magnolia & Tyler,ARHF78W6,7.67,27.60,7.05,33.911928,-117.461394
2021-03-02 12:47:51.159000-08:00,Iowa,Iowa & University,AY3HFT4R,7.81,26.31,7.96,33.975591,-117.339897
2021-03-02 12:50:54.283000-08:00,Chicago,Chicago & University,AGJ1PPLJ,8.84,25.72,7.94,33.975566,-117.348524
2021-03-02 12:56:17.661000-08:00,Cranford,Cranford & University,A4T789S6,16.63,26.95,8.66,33.975624,-117.344182
...,...,...,...,...,...,...,...,...
2021-07-10 23:55:03.070000-07:00,Chicago,Chicago & University,AGJ1PPLJ,9.01,25.78,41.70,33.975566,-117.348524
2021-07-10 23:57:16.538000-07:00,Magnolia,Magnolia & Tyler,ARHF78W6,10.61,25.62,41.46,33.911928,-117.461394
2021-07-10 23:57:30.107000-07:00,Cranford,Cranford & University,A4T789S6,8.43,25.46,42.16,33.975624,-117.344182


In [47]:
clarity_historical_pm25 = historical_clarity.get_PM25()
clarity_historical_pm25.info()
clarity_historical_pm25.head(len(clarity_historical_pm25))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 88847 entries, 2020-09-29 12:05:00-07:00 to 2021-06-22 23:58:00-07:00
Data columns (total 8 columns):
 #   Column                                                          Non-Null Count  Dtype  
---  ------                                                          --------------  -----  
 0   Sensor Name                                                     88847 non-null  object 
 1   Intersection                                                    88847 non-null  object 
 2   Device ID                                                       88847 non-null  object 
 3   PM2.5 Highest Resolution Mass Concentration Calibrated [ug/m3]  88847 non-null  float64
 4   Temperature Internal Highest Resolution [degC]                  88847 non-null  float64
 5   Rel. Humidity Internal Highest Resolution [%]                   88847 non-null  float64
 6   Latitude                                                        88847 non-null  float64
 7   Lon

,Sensor Name,Intersection,Device ID,PM2.5 Highest Resolution Mass Concentration Calibrated [ug/m3],Temperature Internal Highest Resolution [degC],Rel. Humidity Internal Highest Resolution [%],Latitude,Longitude
datetime-America/Los_Angeles,,,,,,,,
2020-09-29 12:05:00-07:00,Magnolia,Magnolia & Tyler,ARHF78W6,3.573135,34.912262,25.265888,33.911928,-117.461394
2020-09-29 13:06:00-07:00,Magnolia,Magnolia & Tyler,ARHF78W6,2.654986,38.290230,22.885481,33.911928,-117.461394
2020-09-29 13:24:00-07:00,Magnolia,Magnolia & Tyler,ARHF78W6,2.318490,38.768219,22.088961,33.911928,-117.461394
2020-09-29 13:41:00-07:00,Magnolia,Magnolia & Tyler,ARHF78W6,2.023389,38.981842,23.118944,33.911928,-117.461394
2020-09-29 13:58:00-07:00,Magnolia,Magnolia & Tyler,ARHF78W6,1.755854,39.166092,21.947052,33.911928,-117.461394
...,...,...,...,...,...,...,...,...
2021-06-22 23:40:00-07:00,Chicago,Chicago & University,AGJ1PPLJ,7.240612,23.197525,52.887772,33.975566,-117.348524
2021-06-22 23:44:00-07:00,Magnolia,Magnolia & Tyler,ARHF78W6,6.212309,22.420464,56.778820,33.911928,-117.461394
2021-06-22 23:50:00-07:00,Iowa,Iowa & University,AY3HFT4R,6.210450,23.152130,52.457466,33.975591,-117.339897


In [48]:
clarity_pm25 = pd.concat([clarity_historical_pm25, clarity_HighRes_pm25], sort=True)
clarity_pm25.info()
clarity_pm25.head(len(clarity_pm25))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 152125 entries, 2020-09-29 12:05:00-07:00 to 2021-07-10 23:58:39.211000-07:00
Data columns (total 8 columns):
 #   Column                                                          Non-Null Count   Dtype  
---  ------                                                          --------------   -----  
 0   Device ID                                                       152125 non-null  object 
 1   Intersection                                                    152125 non-null  object 
 2   Latitude                                                        152125 non-null  float64
 3   Longitude                                                       152125 non-null  float64
 4   PM2.5 Highest Resolution Mass Concentration Calibrated [ug/m3]  152125 non-null  float64
 5   Rel. Humidity Internal Highest Resolution [%]                   152125 non-null  float64
 6   Sensor Name                                                     152125 non-null 

,Device ID,Intersection,Latitude,Longitude,PM2.5 Highest Resolution Mass Concentration Calibrated [ug/m3],Rel. Humidity Internal Highest Resolution [%],Sensor Name,Temperature Internal Highest Resolution [degC]
datetime-America/Los_Angeles,,,,,,,,
2020-09-29 12:05:00-07:00,ARHF78W6,Magnolia & Tyler,33.911928,-117.461394,3.573135,25.265888,Magnolia,34.912262
2020-09-29 13:06:00-07:00,ARHF78W6,Magnolia & Tyler,33.911928,-117.461394,2.654986,22.885481,Magnolia,38.290230
2020-09-29 13:24:00-07:00,ARHF78W6,Magnolia & Tyler,33.911928,-117.461394,2.318490,22.088961,Magnolia,38.768219
2020-09-29 13:41:00-07:00,ARHF78W6,Magnolia & Tyler,33.911928,-117.461394,2.023389,23.118944,Magnolia,38.981842
2020-09-29 13:58:00-07:00,ARHF78W6,Magnolia & Tyler,33.911928,-117.461394,1.755854,21.947052,Magnolia,39.166092
...,...,...,...,...,...,...,...,...
2021-07-10 23:55:03.070000-07:00,AGJ1PPLJ,Chicago & University,33.975566,-117.348524,9.010000,41.700000,Chicago,25.780000
2021-07-10 23:57:16.538000-07:00,ARHF78W6,Magnolia & Tyler,33.911928,-117.461394,10.610000,41.460000,Magnolia,25.620000
2021-07-10 23:57:30.107000-07:00,A4T789S6,Cranford & University,33.975624,-117.344182,8.430000,42.160000,Cranford,25.460000


***
> ### __Clarity PM2.5 Resampling__
***

In [49]:
grouping = ['Sensor Name', 'Intersection', 'Device ID', 'Latitude', 'Longitude']
clarity_pm25_interp = clarity_pm25.groupby(grouping, as_index=True).resample(
    '15T').interpolate('linear').drop(grouping, axis=1).reset_index(drop=False).dropna()
clarity_pm25_interp.info()
clarity_pm25_interp

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109212 entries, 3 to 109214
Data columns (total 9 columns):
 #   Column                                                          Non-Null Count   Dtype                              
---  ------                                                          --------------   -----                              
 0   Sensor Name                                                     109212 non-null  object                             
 1   Intersection                                                    109212 non-null  object                             
 2   Device ID                                                       109212 non-null  object                             
 3   Latitude                                                        109212 non-null  float64                            
 4   Longitude                                                       109212 non-null  float64                            
 5   datetime-America/Los_Angeles   

,Sensor Name,Intersection,Device ID,Latitude,Longitude,datetime-America/Los_Angeles,PM2.5 Highest Resolution Mass Concentration Calibrated [ug/m3],Rel. Humidity Internal Highest Resolution [%],Temperature Internal Highest Resolution [degC]
3,Chicago,Chicago & University,AGJ1PPLJ,33.975566,-117.348524,2020-09-29 15:45:00-07:00,4.076105,23.100634,37.494469
4,Chicago,Chicago & University,AGJ1PPLJ,33.975566,-117.348524,2020-09-29 16:00:00-07:00,4.291387,23.124476,37.366627
5,Chicago,Chicago & University,AGJ1PPLJ,33.975566,-117.348524,2020-09-29 16:15:00-07:00,4.506669,23.148318,37.238785
6,Chicago,Chicago & University,AGJ1PPLJ,33.975566,-117.348524,2020-09-29 16:30:00-07:00,4.721952,23.172160,37.110943
7,Chicago,Chicago & University,AGJ1PPLJ,33.975566,-117.348524,2020-09-29 16:45:00-07:00,4.937234,23.196002,36.983101
...,...,...,...,...,...,...,...,...,...
109210,Magnolia,Magnolia & Tyler,ARHF78W6,33.911928,-117.461394,2021-07-10 22:45:00-07:00,6.286515,50.905624,24.070724
109211,Magnolia,Magnolia & Tyler,ARHF78W6,33.911928,-117.461394,2021-07-10 23:00:00-07:00,6.286515,50.905624,24.070724
109212,Magnolia,Magnolia & Tyler,ARHF78W6,33.911928,-117.461394,2021-07-10 23:15:00-07:00,6.286515,50.905624,24.070724
109213,Magnolia,Magnolia & Tyler,ARHF78W6,33.911928,-117.461394,2021-07-10 23:30:00-07:00,6.286515,50.905624,24.070724


***
> ## __PM2.5 Grouping__
***

In [50]:
pm25_pivot = pd.pivot_table(clarity_pm25_interp,
                            values='PM2.5 Highest Resolution Mass Concentration Calibrated [ug/m3]',
                            index='datetime-America/Los_Angeles', columns='Sensor Name')
pm25_pivot.rename(columns={
    'Chicago' : 'Chicago PM2.5 [ug/m3]',
    'Cranford': 'Cranford PM2.5 [ug/m3]',
    'Iowa'    : 'Iowa PM2.5 [ug/m3]',
    'Magnolia': 'Magnolia PM2.5 [ug/m3]'
}, inplace=True)
pm25_pivot.dropna(inplace=True)
if SAVE_TABLES:
    pm25_pivot.to_csv('../data/clarity_data/pm25_readings.csv')
pm25_pivot.info()
pm25_pivot

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 27297 entries, 2020-09-29 15:45:00-07:00 to 2021-07-10 23:45:00-07:00
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Chicago PM2.5 [ug/m3]   27297 non-null  float64
 1   Cranford PM2.5 [ug/m3]  27297 non-null  float64
 2   Iowa PM2.5 [ug/m3]      27297 non-null  float64
 3   Magnolia PM2.5 [ug/m3]  27297 non-null  float64
dtypes: float64(4)
memory usage: 1.0 MB


Sensor Name,Chicago PM2.5 [ug/m3],Cranford PM2.5 [ug/m3],Iowa PM2.5 [ug/m3],Magnolia PM2.5 [ug/m3]
datetime-America/Los_Angeles,,,,
2020-09-29 15:45:00-07:00,4.076105,8.111194,12.761862,3.818308
2020-09-29 16:00:00-07:00,4.291387,8.112177,12.765333,3.966438
2020-09-29 16:15:00-07:00,4.506669,8.113160,12.768803,4.114569
2020-09-29 16:30:00-07:00,4.721952,8.114143,12.772273,4.262699
2020-09-29 16:45:00-07:00,4.937234,8.115126,12.775744,4.410830
...,...,...,...,...
2021-07-10 22:45:00-07:00,8.100380,12.734099,2.956193,6.286515
2021-07-10 23:00:00-07:00,8.101363,12.737570,2.954288,6.286515
2021-07-10 23:15:00-07:00,8.102346,12.741040,2.952383,6.286515


***
> ## __NO2 Data__
***

In [55]:
historical_clarity_no2 = pd.read_csv(fpath_no2)
historical_clarity_no2.info()
historical_clarity_no2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509963 entries, 0 to 509962
Data columns (total 42 columns):
 #   Column                                                      Non-Null Count   Dtype  
---  ------                                                      --------------   -----  
 0   Site ID                                                     509963 non-null  object 
 1   Longitude                                                   509963 non-null  float64
 2   Latitude                                                    509963 non-null  float64
 3   Location Type                                               509963 non-null  object 
 4   Date                                                        509963 non-null  object 
 5   Average                                                     509963 non-null  object 
 6   VOC                                                         35719 non-null   float64
 7   VOC - Raw                                                   35719 non-null

,Site ID,Longitude,Latitude,Location Type,Date,Average,VOC,VOC - Raw,Relative Humidity,Relative Humidity - Raw,Temperature,Temperature - Raw,Particulate Matter (2.5) Number Concentration,Particulate Matter (2.5) Number Concentration - Calibrated,Particulate Matter (2.5) Number Concentration - Raw,Particulate Matter (2.5) Mass Concentration,Particulate Matter (2.5) Mass Concentration - AQI,Particulate Matter (2.5) Mass Concentration - EPA NowCast,Particulate Matter (2.5) Mass Concentration - Calibrated,Particulate Matter (2.5) Mass Concentration - Raw,Particulate Matter (1) Number Concentration,Particulate Matter (1) Number Concentration - Calibrated,Particulate Matter (1) Number Concentration - Raw,Particulate Matter (1) Mass Concentration,Particulate Matter (1) Mass Concentration - Calibrated,Particulate Matter (1) Mass Concentration - Raw,Particulate Matter (10) Number Concentration,Particulate Matter (10) Number Concentration - Calibrated,Particulate Matter (10) Number Concentration - Raw,Particulate Matter (10) Mass Concentration,Particulate Matter (10) Mass Concentration - Calibrated,Particulate Matter (10) Mass Concentration - Raw,Nitrogen Dioxide,Nitrogen Dioxide - AQI,Carbon Dioxide,Carbon Dioxide - Calibrated,Carbon Dioxide - Raw,datasourceType,outputFrequency,Data Source,Nitrogen Dioxide - Calibrated,Nitrogen Dioxide - Raw
0,AY3HFT4R,-117.261331,33.980781,Point,2020-09-30 11:57:55,Base,NaN,NaN,25.09,25.09,27.35,27.35,11.86,NaN,11.86,10.12,NaN,NaN,NaN,10.12,11.01,NaN,11.01,5.00,NaN,5.00,11.94,NaN,11.94,14.00,NaN,14.00,4.123432,NaN,NaN,NaN,NaN,NaN,NaN,Clarity,4.123432,-0.472099
1,ARHF78W6,-117.261331,33.980781,Point,2020-09-30 11:57:20,Base,NaN,NaN,26.62,26.62,26.43,26.43,10.59,NaN,10.59,8.20,NaN,NaN,NaN,8.20,10.04,NaN,10.04,4.59,NaN,4.59,10.70,NaN,10.70,11.06,NaN,11.06,12.438928,NaN,NaN,NaN,NaN,NaN,NaN,Clarity,12.438928,1.582969
2,A4T789S6,-117.261331,33.980781,Point,2020-09-30 11:57:11,Base,NaN,NaN,26.49,26.49,26.77,26.77,11.61,NaN,11.61,9.58,NaN,NaN,NaN,9.58,10.93,NaN,10.93,5.00,NaN,5.00,11.74,NaN,11.74,11.88,NaN,11.88,25.850752,NaN,NaN,NaN,NaN,NaN,NaN,Clarity,25.850752,-16.697574
3,AQ7GDKW8,-117.261331,33.980781,Point,2020-09-30 11:56:15,Base,NaN,NaN,27.30,27.30,26.74,26.74,13.48,NaN,13.48,10.55,NaN,NaN,NaN,10.55,12.80,NaN,12.80,6.00,NaN,6.00,13.59,NaN,13.59,12.17,NaN,12.17,13.446313,NaN,NaN,NaN,NaN,NaN,NaN,Clarity,13.446313,-0.003394
4,AGJ1PPLJ,-117.261331,33.980781,Point,2020-09-30 11:50:54,Base,NaN,NaN,28.44,28.44,26.73,26.73,12.39,NaN,12.39,10.31,NaN,NaN,NaN,10.31,11.80,NaN,11.80,5.58,NaN,5.58,12.50,NaN,12.50,11.08,NaN,11.08,8.055877,NaN,NaN,NaN,NaN,NaN,NaN,Clarity,8.055877,3.317820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509958,AY3HFT4R,-117.339897,33.975591,Point,2021-12-02 12:02:27,Base,NaN,NaN,40.57,40.57,12.74,12.74,17.00,NaN,17.00,21.03,NaN,NaN,21.03,17.78,15.80,NaN,15.80,7.78,NaN,7.78,17.33,NaN,17.33,31.55,NaN,31.55,65.780263,NaN,NaN,NaN,NaN,NaN,NaN,Clarity,65.780263,49.770931
509959,AQ7GDKW8,-117.617220,34.030830,Point,2021-12-02 12:01:06,Base,NaN,NaN,21.06,21.06,23.46,23.46,7.28,NaN,7.28,10.45,NaN,NaN,10.45,6.23,6.90,NaN,6.90,2.81,NaN,2.81,7.34,NaN,7.34,7.08,NaN,7.08,43.535053,NaN,NaN,NaN,NaN,NaN,NaN,Clarity,43.535053,22.302955
509960,A4T789S6,-117.344182,33.975624,Point,2021-12-02 12:01:02,Base,NaN,NaN,43.77,43.77,11.96,11.96,19.31,NaN,19.31,17.77,NaN,NaN,17.77,19.27,17.94,NaN,17.94,10.23,NaN,10.23,19.50,NaN,19.50,24.65,NaN,24.65,37.855784,NaN,NaN,NaN,NaN,NaN,NaN,Clarity,37.855784,40.378503
509961,AXD4VGR2,-117.617220,34.030830,Point,2021-12-02 12:00:59,Base,NaN,NaN,21.02,21.02,23.40,23.40,6.39,NaN,6.39,9.48,NaN,NaN,9.48,3.00,6.06,NaN,6.06,1.61,NaN,1.61,6.39,NaN,6.39,7.85,NaN,7.85,38.955826,NaN,NaN,NaN,NaN,NaN,NaN,Clarity,38.955826,22.725816


In [56]:
historical_clarity_no2 = ic.historical_no2(historical_clarity_no2)
clarity_no2_data = historical_clarity_no2.get_data()
clarity_no2_data

,Sensor Name,Site ID,Nitrogen Dioxide
datetime-America/Los_Angeles,,,
2020-09-30 04:57:55-07:00,Iowa,AY3HFT4R,4.123432
2020-09-30 04:57:20-07:00,Magnolia,ARHF78W6,12.438928
2020-09-30 04:57:11-07:00,Cranford,A4T789S6,25.850752
2020-09-30 04:50:54-07:00,Chicago,AGJ1PPLJ,8.055877
2020-09-30 04:40:45-07:00,Iowa,AY3HFT4R,5.536795
...,...,...,...
2021-12-02 04:04:29-08:00,Chicago,AGJ1PPLJ,50.801184
2021-12-02 04:02:54-08:00,Magnolia,ARHF78W6,42.141093
2021-12-02 04:02:27-08:00,Iowa,AY3HFT4R,65.780263


***
>> ### __Interpolating NO2 Data__
Method: Resampling to 15 Minute intervals with linear interpolation
***

In [65]:
no2_grouping = ['Sensor Name', 'Site ID']
clarity_no2_interp = clarity_no2_data.groupby(no2_grouping, as_index=True).resample('15T').interpolate('linear').drop(
    no2_grouping, axis=1).reset_index(drop=False).dropna()
clarity_no2_interp.info()
clarity_no2_interp

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 164114 entries, 2020-10-06 04:30:00-07:00 to 2021-12-02 15:45:00-08:00
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Sensor Name       164114 non-null  object 
 1   Site ID           164114 non-null  object 
 2   Nitrogen Dioxide  164114 non-null  float64
dtypes: float64(1), object(2)
memory usage: 5.0+ MB


,Sensor Name,Site ID,Nitrogen Dioxide
datetime-America/Los_Angeles,,,
2020-10-06 04:30:00-07:00,Chicago,AGJ1PPLJ,12.642770
2020-10-06 04:45:00-07:00,Chicago,AGJ1PPLJ,12.641437
2020-10-06 05:00:00-07:00,Chicago,AGJ1PPLJ,12.640103
2020-10-06 05:15:00-07:00,Chicago,AGJ1PPLJ,12.638770
2020-10-06 05:30:00-07:00,Chicago,AGJ1PPLJ,12.637437
...,...,...,...
2021-12-02 14:45:00-08:00,Magnolia,ARHF78W6,72.918292
2021-12-02 15:00:00-08:00,Magnolia,ARHF78W6,72.918292
2021-12-02 15:15:00-08:00,Magnolia,ARHF78W6,72.918292


In [66]:
no2_pivot = pd.pivot_table(clarity_no2_interp,
                           values='Nitrogen Dioxide',
                           index='datetime-America/Los_Angeles',
                           columns='Sensor Name')
no2_pivot.rename(columns={
    'Chicago' : 'Chicago NO2',
    'Cranford': 'Cranford NO2',
    'Iowa'    : 'Iowa NO2',
    'Magnolia': 'Magnolia NO2'
}, inplace=True)
if SAVE_TABLES:
    no2_pivot.to_csv('../data/clarity_data/no2_readings.csv')
no2_pivot.info()
no2_pivot

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 41184 entries, 2020-09-29 17:00:00-07:00 to 2021-12-02 15:45:00-08:00
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Chicago NO2   40562 non-null  float64
 1   Cranford NO2  41184 non-null  float64
 2   Iowa NO2      41184 non-null  float64
 3   Magnolia NO2  41184 non-null  float64
dtypes: float64(4)
memory usage: 1.6 MB


Sensor Name,Chicago NO2,Cranford NO2,Iowa NO2,Magnolia NO2
datetime-America/Los_Angeles,,,,
2020-09-29 17:00:00-07:00,NaN,37.685778,107.463278,65.521058
2020-09-29 17:15:00-07:00,NaN,37.683551,107.407847,65.500511
2020-09-29 17:30:00-07:00,NaN,37.681324,107.352416,65.479965
2020-09-29 17:45:00-07:00,NaN,37.679097,107.296986,65.459418
2020-09-29 18:00:00-07:00,NaN,37.676870,107.241555,65.438872
...,...,...,...,...
2021-12-02 14:45:00-08:00,37.696914,107.740431,65.623790,72.918292
2021-12-02 15:00:00-08:00,37.694687,107.685001,65.603243,72.918292
2021-12-02 15:15:00-08:00,37.692460,107.629570,65.582697,72.918292


***
# __Meteorological Data__
***

***
> ## __Weather Data__
***

In [23]:
openWeather = pd.read_csv(fPath_weatherData)
openWeather = ic.OpenWeather(openWeather)
openWeather_data = openWeather.get_data()

openWeather_data.info()
openWeather_data.head(len(openWeather_data))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10145 entries, 2020-08-31 17:00:00-07:00 to 2021-10-22 16:00:00-07:00
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   temp        10145 non-null  float64
 1   pressure    10145 non-null  int64  
 2   humidity    10145 non-null  int64  
 3   wind_speed  10145 non-null  float64
 4   wind_deg    10145 non-null  int64  
dtypes: float64(2), int64(3)
memory usage: 475.5 KB


,temp,pressure,humidity,wind_speed,wind_deg
datetime-America/Los_Angeles,,,,,
2020-08-31 17:00:00-07:00,31.07,1009,40,2.24,220
2020-08-31 18:00:00-07:00,28.35,1009,45,2.24,233
2020-08-31 19:00:00-07:00,24.96,1010,54,1.34,219
2020-08-31 20:00:00-07:00,22.89,1011,62,0.89,186
2020-08-31 21:00:00-07:00,21.48,1012,65,1.34,170
...,...,...,...,...,...
2021-10-22 12:00:00-07:00,22.62,1016,42,1.34,277
2021-10-22 13:00:00-07:00,24.08,1015,37,1.79,224
2021-10-22 14:00:00-07:00,23.92,1014,40,2.24,260


In [24]:
openWeather_data_interp = openWeather_data[~openWeather_data.index.duplicated()].resample('15T').ffill()
openWeather_data_interp = openWeather_data_interp[~openWeather_data_interp.index.duplicated()]
if SAVE_TABLES:
    openWeather_data_interp.to_csv('../data/weather_data/OpenWeather_15min.csv')

openWeather_data_interp.info()
openWeather_data_interp.head(len(openWeather_data))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 40029 entries, 2020-08-31 17:00:00-07:00 to 2021-10-22 16:00:00-07:00
Freq: 15T
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   temp        40029 non-null  float64
 1   pressure    40029 non-null  int64  
 2   humidity    40029 non-null  int64  
 3   wind_speed  40029 non-null  float64
 4   wind_deg    40029 non-null  int64  
dtypes: float64(2), int64(3)
memory usage: 1.8 MB


,temp,pressure,humidity,wind_speed,wind_deg
datetime-America/Los_Angeles,,,,,
2020-08-31 17:00:00-07:00,31.07,1009,40,2.24,220
2020-08-31 17:15:00-07:00,31.07,1009,40,2.24,220
2020-08-31 17:30:00-07:00,31.07,1009,40,2.24,220
2020-08-31 17:45:00-07:00,31.07,1009,40,2.24,220
2020-08-31 18:00:00-07:00,28.35,1009,45,2.24,233
...,...,...,...,...,...
2020-12-15 07:00:00-08:00,8.42,1023,52,0.45,317
2020-12-15 07:15:00-08:00,8.42,1023,52,0.45,317
2020-12-15 07:30:00-08:00,8.42,1023,52,0.45,317


***
# __Traffic Density__
***
$$v:\text{average speed }(\frac{miles}{hour})$$
$$k:\text{density }(\frac{\text{# of vehicles}}{miles})$$
$$q=kv:\text{Traffic Flow }(\frac{\text{# of vehicles}}{hour})$$
Rearranging for k gives us :
$$k = \frac{q}{v}$$


In [25]:
miles_per_meter = 0.0006213712
path_length_m = {
    'Local 1' : 836,
    'Local 2' : 893,
    'Local 9' : 918,
    'Local 10': 836,
    'Fwy 1'   : 2326,
    'Fwy 2'   : 2508,
    'Fwy 3'   : 2651,
    'Fwy 4'   : 2450
}

***
> ## __GridSmart Data__
***

In [26]:
gs_files = os.listdir(FPath_GridSmart_csv)
for i in nLoop(gs_files):
    gs_files[i] = os.path.join(FPath_GridSmart_csv, gs_files[i])
full_table = pd.read_csv(gs_files[0])
gridsmart = ic.GridSmart_csv(full_table)

for i in range(1, len(gs_files)):
    table = pd.read_csv(gs_files[i])
    gridsmart.add_data(table)

gs_data = gridsmart.get_data()
gs_data.info()
gs_data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7783 entries, 2021-10-10 00:00:00-07:00 to 2021-06-09 23:45:00-07:00
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Eastbound  7783 non-null   int32
 1   Westbound  7783 non-null   int32
dtypes: int32(2)
memory usage: 121.6 KB


,Eastbound,Westbound
datetime-America/Los_Angeles,,
2021-10-10 00:00:00-07:00,57,42
2021-10-10 00:15:00-07:00,42,37
2021-10-10 00:30:00-07:00,43,30
2021-10-10 00:45:00-07:00,37,26
2021-10-10 01:00:00-07:00,36,34
...,...,...
2021-06-09 22:45:00-07:00,60,39
2021-06-09 23:00:00-07:00,51,57
2021-06-09 23:15:00-07:00,55,34


In [27]:
gs_data.rename(columns={
    'Northbound': 'GS-NB # of Vehicles/15 min',
    'Eastbound' : 'GS-EB # of Vehicles/15 min',
    'Southbound': 'GS-SB # of Vehicles/15 min',
    'Westbound' : 'GS-WB # of Vehicles/15 min'
}, inplace=True)
gs_data.to_csv('../data/gridsmart/gridsmart_processed.csv')
gs_data.info()
gs_data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7783 entries, 2021-10-10 00:00:00-07:00 to 2021-06-09 23:45:00-07:00
Data columns (total 2 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   GS-EB # of Vehicles/15 min  7783 non-null   int32
 1   GS-WB # of Vehicles/15 min  7783 non-null   int32
dtypes: int32(2)
memory usage: 121.6 KB


,GS-EB # of Vehicles/15 min,GS-WB # of Vehicles/15 min
datetime-America/Los_Angeles,,
2021-10-10 00:00:00-07:00,57,42
2021-10-10 00:15:00-07:00,42,37
2021-10-10 00:30:00-07:00,43,30
2021-10-10 00:45:00-07:00,37,26
2021-10-10 01:00:00-07:00,36,34
...,...,...
2021-06-09 22:45:00-07:00,60,39
2021-06-09 23:00:00-07:00,51,57
2021-06-09 23:15:00-07:00,55,34


***
> ## __PEMS Freeway Data__
***
PEMS Data gives Vehicle Miles Traveled (VMT) which is flow times the length traveled.
$$VMT = qL$$

Rearranging gives us,
$$q=\frac{VMY}{L}$$
$$k=\frac{VMT}{L*v}$$

PEMS freeway data is the aggregated sum of VMT for the hour. Upscaling resamples that sum and spaces it over 15 minutes intervals.



In [28]:
pems_length_miles = 5.3  # miles

In [29]:
pems_north = util.excel_folder2table(FPath_PEMS + "/I215_northbound/", 'Report Data', axis=1)
pems_north = pems_north.iloc[:, ~pems_north.columns.duplicated()]
pems_south = util.excel_folder2table(FPath_PEMS + "/I215_southbound/", 'Report Data', axis=1)
pems_south = pems_south.iloc[:, ~pems_south.columns.duplicated()]
pems = ic.PEMS(pems_north, pems_south)
pems_data = pems.get_data()
pems_data.info()
pems_data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7295 entries, 2021-01-01 00:00:00-08:00 to 2021-10-31 23:00:00-07:00
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Northbound (VMT)  7295 non-null   float64
 1   Southbound (VMT)  7295 non-null   float64
dtypes: float64(2)
memory usage: 171.0 KB


,Northbound (VMT),Southbound (VMT)
datetime-America/Los_Angeles,,
2021-01-01 00:00:00-08:00,9490.9,10054.1
2021-01-01 01:00:00-08:00,10706.2,10804.1
2021-01-01 02:00:00-08:00,8224.8,8311.5
2021-01-01 03:00:00-08:00,6977.0,6752.9
2021-01-01 04:00:00-08:00,7186.1,6874.1
...,...,...
2021-10-31 19:00:00-07:00,30163.2,30002.5
2021-10-31 20:00:00-07:00,32028.4,30913.0
2021-10-31 21:00:00-07:00,33554.3,31339.8


In [30]:
pems_data_15min = pems_data/4
pems_data_15min = pems_data_15min.resample('15T').ffill()
if SAVE_TABLES:
    pems_data_15min.to_csv('../data/pems/PEMS_I215_VMT_15min.csv')
pems_data_15min.info()
pems_data_15min

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29177 entries, 2021-01-01 00:00:00-08:00 to 2021-10-31 23:00:00-07:00
Freq: 15T
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Northbound (VMT)  29177 non-null  float64
 1   Southbound (VMT)  29177 non-null  float64
dtypes: float64(2)
memory usage: 683.8 KB


,Northbound (VMT),Southbound (VMT)
datetime-America/Los_Angeles,,
2021-01-01 00:00:00-08:00,2372.725,2513.525
2021-01-01 00:15:00-08:00,2372.725,2513.525
2021-01-01 00:30:00-08:00,2372.725,2513.525
2021-01-01 00:45:00-08:00,2372.725,2513.525
2021-01-01 01:00:00-08:00,2676.550,2701.025
...,...,...
2021-10-31 22:00:00-07:00,6576.600,6192.000
2021-10-31 22:15:00-07:00,6576.600,6192.000
2021-10-31 22:30:00-07:00,6576.600,6192.000


***
> ## __Travel Time__
***

In [31]:
tt_file_list = os.listdir(FPath_travelTime_csv)
for i in util.nLoop(tt_file_list):
    tt_file_list[i] = FPath_travelTime_csv + "\\" + tt_file_list[i]
tt_csvData = ic.readTravelTimeCSV(tt_file_list[0])
for fpath in range(1, len(tt_file_list)):
    temp = ic.readTravelTimeCSV(tt_file_list[fpath])
    tt_csvData = pd.concat([tt_csvData, temp], axis=0)
tt_csvData.info()
tt_csvData

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6612 entries, 0 to 559
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DT        6612 non-null   object
 1   Local 1   6612 non-null   object
 2   Local 2   6612 non-null   object
 3   Local 3   117 non-null    object
 4   Local 4   117 non-null    object
 5   Local 5   117 non-null    object
 6   Local 6   117 non-null    object
 7   Local 7   117 non-null    object
 8   Local 8   117 non-null    object
 9   Local 9   6612 non-null   object
 10  Local 10  6612 non-null   object
 11  Fwy 1     6612 non-null   object
 12  Fwy 3     6612 non-null   object
 13  Fwy 2     6612 non-null   object
 14  Fwy 4     6612 non-null   object
dtypes: object(15)
memory usage: 826.5+ KB


,DT,Local 1,Local 2,Local 3,Local 4,Local 5,Local 6,Local 7,Local 8,Local 9,Local 10,Fwy 1,Fwy 3,Fwy 2,Fwy 4
0,2021-04-21 14:00,143,140,111,46,115,100,51,79,111,154,257,144,247,298
1,2021-04-21 14:15,127,151,98,46,123,116,46,70,116,151,259,143,257,291
2,2021-04-21 14:30,137,142,75,48,118,122,47,71,123,140,255,149,249,292
3,2021-04-21 14:45,138,135,77,45,117,113,48,81,122,147,261,155,246,354
4,2021-04-21 15:00,141,142,75,43,125,114,46,82,113,149,266,148,232,322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,2021-07-14 16:30,145,169,NaN,NaN,NaN,NaN,NaN,NaN,123,148,241,160,272,328
556,2021-07-14 16:45,134,143,NaN,NaN,NaN,NaN,NaN,NaN,125,143,247,151,302,329
557,2021-07-14 17:00,145,162,NaN,NaN,NaN,NaN,NaN,NaN,121,166,243,164,260,320
558,2021-07-14 17:15,135,145,NaN,NaN,NaN,NaN,NaN,NaN,133,146,246,167,295,376


In [32]:
travel_time = ic.travel_time(tt_csvData)
tt_data = travel_time.get_data()
tt_data = tt_data[~tt_data.index.duplicated()]
if SAVE_TABLES:
    tt_data.to_csv('../data/travel_time/travel_time.csv')

tt_data.info()
tt_data.head(len(tt_data))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6612 entries, 2021-04-21 14:00:00-07:00 to 2021-07-14 17:30:00-07:00
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Local 1 (mph)   6612 non-null   float64
 1   Local 2 (mph)   6612 non-null   float64
 2   Local 9 (mph)   6612 non-null   float64
 3   Local 10 (mph)  6612 non-null   float64
 4   Fwy 1 (mph)     6612 non-null   float64
 5   Fwy 3 (mph)     6612 non-null   float64
 6   Fwy 2 (mph)     6612 non-null   float64
 7   Fwy 4 (mph)     6612 non-null   float64
dtypes: float64(8)
memory usage: 464.9 KB


,Local 1 (mph),Local 2 (mph),Local 9 (mph),Local 10 (mph),Fwy 1 (mph),Fwy 3 (mph),Fwy 2 (mph),Fwy 4 (mph)
datetime-America/Los_Angeles,,,,,,,,
2021-04-21 14:00:00-07:00,13.077474,14.268458,18.500068,12.143368,20.245579,41.181376,22.713507,18.390919
2021-04-21 14:15:00-07:00,14.725029,13.229034,17.702651,12.384627,20.089243,41.469357,21.829713,18.833312
2021-04-21 14:30:00-07:00,13.650210,14.067494,16.695183,13.357705,20.404368,39.799450,22.531069,18.768815
2021-04-21 14:45:00-07:00,13.551295,14.796919,16.832029,12.721624,19.935302,38.258827,22.805838,15.481621
2021-04-21 15:00:00-07:00,13.262970,14.067494,18.172633,12.550864,19.560578,40.068366,24.182053,17.020167
...,...,...,...,...,...,...,...,...
2021-07-14 16:30:00-07:00,12.897095,11.820024,16.695183,12.635667,21.589684,37.063238,20.625868,16.708823
2021-07-14 16:45:00-07:00,13.955811,13.969120,16.428060,13.077474,21.065238,39.272305,18.576941,16.658036
2021-07-14 17:00:00-07:00,12.897095,12.330766,16.971136,11.265535,21.411991,36.159257,21.577832,17.126543


***
> ## __Calculating Traffic Density__
***
Similar road conditions exist for local 1 and 2, as well as local 3 and 4 so we assume gridsmart sensor counts can apply to both paths

In [33]:
traffic_density = pd.merge(tt_data, gs_data, left_index=True, right_index=True)
traffic_density = pd.merge(traffic_density, pems_data_15min, left_index=True, right_index=True)
traffic_density = traffic_density[(traffic_density > 0).all(1)]
traffic_density['Density Local 1 (#Vehicles/mile)'] = (
        (traffic_density['GS-WB # of Vehicles/15 min']*4)
        /traffic_density['Local 1 (mph)']
)
traffic_density['Density Local 2 (#Vehicles/mile)'] = (
        (traffic_density['GS-WB # of Vehicles/15 min']*4)
        /traffic_density['Local 2 (mph)']
)
traffic_density['Density Local 9 (#Vehicles/mile)'] = (
        (traffic_density['GS-EB # of Vehicles/15 min']*4)
        /traffic_density['Local 9 (mph)']
)
traffic_density['Density Local 10 (#Vehicles/mile)'] = (
        (traffic_density['GS-EB # of Vehicles/15 min']*4)
        /traffic_density['Local 10 (mph)']
)

traffic_density['Density Fwy 1 (#Vehicles/mile)'] = (
        traffic_density['Northbound (VMT)']
        /pems_length_miles*traffic_density['Fwy 4 (mph)']
)
traffic_density['Density Fwy 2 (#Vehicles/mile)'] = (
        traffic_density['Southbound (VMT)']
        /pems_length_miles*traffic_density['Fwy 4 (mph)']
)
traffic_density.rename(columns={
    'Northbound (VMT)': 'PEMS Northbound (VMT)',
    'Southbound (VMT)': 'PEMS Southbound (VMT)',
}, inplace=True)
traffic_density = traffic_density.iloc[:, -6:]
if SAVE_TABLES:
    traffic_density.to_csv('../data/traffic_density/traffic_density.csv')
traffic_density.info()
traffic_density

SyntaxError: invalid syntax (Temp/ipykernel_18064/474273068.py, line 4)

***
# __Final Dataset__
***

In [ ]:
final_pm25_TrafficDensity = pm25_pivot.merge(openWeather_data_interp, left_index=True, right_index=True)
print(f'Data Points: {len(final_pm25_TrafficDensity)} -Merged OpenWeather with Clarity')
final_pm25_TrafficDensity = final_pm25_TrafficDensity.merge(carb_pm25_data_interp, left_index=True, right_index=True)
print(f'Data Points: {len(final_pm25_TrafficDensity)} -Merged Carb')
final_pm25_TrafficDensity = final_pm25_TrafficDensity.merge(traffic_density, left_index=True,
                                                            right_index=True)
print(f'Data Points: {len(final_pm25_TrafficDensity)} -Merged Travel Time')

final_pm25_TrafficDensity.reset_index('datetime-America/Los_Angeles', inplace=True)
final_pm25_TrafficDensity = dtk.df_decomposeDT(final_pm25_TrafficDensity, 0)
final_pm25_TrafficDensity.drop(
    columns=[
        'timezone', 'week of year',
    ],
    inplace=True
)
final_pm25_TrafficDensity.set_index('datetime-America/Los_Angeles', inplace=True)
final_pm25_TrafficDensity = pd.get_dummies(final_pm25_TrafficDensity, columns=['dayofweek'])
col_vals = [
    'Iowa PM2.5 [ug/m3]',
    'Chicago PM2.5 [ug/m3]',
    'Cranford PM2.5 [ug/m3]',
    'Magnolia PM2.5 [ug/m3]',
    'year',
    'month',
    'day',
    'dayofweek_Sunday',
    'dayofweek_Monday',
    'dayofweek_Tuesday',
    'dayofweek_Wednesday',
    'dayofweek_Thursday',
    'dayofweek_Friday',
    'dayofweek_Saturday',
    'temp',
    'pressure',
    'humidity',
    'wind_speed',
    'wind_deg',
    'PM2.5 Background [ug/m3]',
    'Density Local 1 (#Vehicles/mile)',
    'Density Local 2 (#Vehicles/mile)',
    'Density Local 9 (#Vehicles/mile)',
    'Density Local 10 (#Vehicles/mile)',
    'Density Fwy 1 (#Vehicles/mile)',
    'Density Fwy 2 (#Vehicles/mile)',
]
final_pm25_TrafficDensity = final_pm25_TrafficDensity.loc[:, col_vals]
if SAVE_TABLES:
    final_pm25_TrafficDensity.to_csv(r'../data/complete_sets/full_pm25.csv')
final_pm25_TrafficDensity.info()
final_pm25_TrafficDensity

In [ ]:
final_pm25_TravelTime = pm25_pivot.merge(openWeather_data_interp, left_index=True, right_index=True)
print(f'Data Points: {len(final_pm25_TravelTime)} -Merged OpenWeather with Clarity')
final_pm25_TravelTime = final_pm25_TravelTime.merge(carb_pm25_data_interp, left_index=True, right_index=True)
print(f'Data Points: {len(final_pm25_TravelTime)} -Merged Carb')
final_pm25_TravelTime = final_pm25_TravelTime.merge(tt_data, left_index=True, right_index=True)
print(f'Data Points: {len(final_pm25_TravelTime)} -Merged Travel Time')
final_pm25_TravelTime.reset_index('datetime-America/Los_Angeles', inplace=True)
final_pm25_TravelTime = dtk.df_decomposeDT(final_pm25_TravelTime, 0)
final_pm25_TravelTime.drop(
    columns=[
        'timezone', 'week of year',
    ],
    inplace=True
)
final_pm25_TravelTime.set_index('datetime-America/Los_Angeles', inplace=True)
final_pm25_TravelTime = pd.get_dummies(final_pm25_TravelTime, columns=['dayofweek'])
col_vals_2 = [
    'Iowa PM2.5 [ug/m3]',
    'Chicago PM2.5 [ug/m3]',
    'Cranford PM2.5 [ug/m3]',
    'Magnolia PM2.5 [ug/m3]',
    'year',
    'month',
    'day',
    'dayofweek_Sunday',
    'dayofweek_Monday',
    'dayofweek_Tuesday',
    'dayofweek_Wednesday',
    'dayofweek_Thursday',
    'dayofweek_Friday',
    'dayofweek_Saturday',
    'temp',
    'pressure',
    'humidity',
    'wind_speed',
    'wind_deg',
    'PM2.5 Background [ug/m3]',
    'Local 1 (mph)',
    'Local 2 (mph)',
    'Local 9 (mph)',
    'Local 10 (mph)',
    'Fwy 1 (mph)',
    'Fwy 4 (mph)'
]
final_pm25_TravelTime = final_pm25_TravelTime.loc[:, col_vals_2]
if SAVE_TABLES:
    final_pm25_TravelTime.to_csv('../data/complete_sets/full_pm25_TravelTime.csv')

final_pm25_TravelTime.info()
final_pm25_TravelTime

In [ ]:
print(f'Starting Data Points: {len(no2_pivot)}')
final_no2_TrafficDensity = no2_pivot.merge(openWeather_data_interp,
                                           left_index=True, right_index=True)
print(f'    Data Points: {len(final_no2_TrafficDensity)} - Merged OpenWeather with NO2 Data')
final_no2_TrafficDensity = final_no2_TrafficDensity.merge(carb_pm25_data_interp, left_index=True, right_index=True)
print(f'    Data Points: {len(final_no2_TrafficDensity)} - Merged Carb with NO2 Data')
final_no2_TrafficDensity=final_no2_TrafficDensity.merge(traffic_density,
                                                        left_index=True,right_index=True)
print(f'    Data Points: {len(final_no2_TrafficDensity)} - Merged Carb Trraffic Density with NO2 Data')
final_no2_TrafficDensity.reset_index('datetime-America/Los_Angeles',inplace=True)
final_no2_TrafficDensity = dtk.df_decomposeDT(final_no2_TrafficDensity,0)
final_no2_TrafficDensity.drop(
    columns=['timezone','week of year'],
    inplace=True
)
final_no2_TrafficDensity.set_index('datetime-America/Los_Angeles',inplace=True)
final_no2_TrafficDensity = pd.get_dummies(final_no2_TrafficDensity,columns=['dayofweek'])

# TODO: Get NO2 Background Data and Reorder Data

***
## __Split by Sensor__
***

In [ ]:
chicago_drop = ['Cranford PM2.5 [ug/m3]', 'Iowa PM2.5 [ug/m3]', 'Magnolia PM2.5 [ug/m3]']
chicago = final_pm25_TrafficDensity.drop(columns=chicago_drop)
chicago_2 = final_pm25_TravelTime.drop(columns=chicago_drop)
if SAVE_TABLES:
    chicago.to_csv('../data/complete_sets/chicago/chicago_full_pm25.csv')
    chicago_2.to_csv('../data/complete_sets/chicago/chicago_full_pm25_TravelTime.csv')

In [ ]:
cranford_drop = ['Chicago PM2.5 [ug/m3]', 'Iowa PM2.5 [ug/m3]', 'Magnolia PM2.5 [ug/m3]']
cranford = final_pm25_TrafficDensity.drop(columns=cranford_drop)
cranford_2 = final_pm25_TravelTime.drop(columns=cranford_drop)
if SAVE_TABLES:
    cranford.to_csv('../data/complete_sets/cranford/cranford_full_pm25.csv')
    cranford_2.to_csv('../data/complete_sets/cranford/cranford_full_pm25_TravelTime.csv')

In [ ]:
iowa_drop = ['Chicago PM2.5 [ug/m3]', 'Cranford PM2.5 [ug/m3]', 'Magnolia PM2.5 [ug/m3]']
iowa = final_pm25_TrafficDensity.drop(columns=iowa_drop)
iowa_2 = final_pm25_TravelTime.drop(columns=iowa_drop)
if SAVE_TABLES:
    iowa.to_csv('../data/complete_sets/iowa/iowa_full_pm25.csv')
    iowa_2.to_csv('../data/complete_sets/iowa/iowa_full_pm25_TravelTime.csv')

In [ ]:
magnolia_drop = ['Iowa PM2.5 [ug/m3]', 'Cranford PM2.5 [ug/m3]', 'Chicago PM2.5 [ug/m3]']
magnolia = final_pm25_TrafficDensity.drop(columns=magnolia_drop)
magnolia_2 = final_pm25_TravelTime.drop(columns=magnolia_drop)
if SAVE_TABLES:
    magnolia.to_csv('../data/complete_sets/magnolia/magnolia_full_pm25.csv')
    magnolia_2.to_csv('../data/complete_sets/magnolia/magnolia_full_pm25_TravelTime.csv')